# 基于卷积神经网络实现猫狗识别

下载数据集到本地，[链接](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data),下载方法可见 README.md

### 数据导入与预处理

In [4]:
import cv2 
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.preprocessing import image
from PIL import Image
from IPython.display import display
from keras.utils import np_utils
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
C:\Users\yeyiy\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 定义调用模块

### 获取训练数据和验证数据模块

In [62]:
TRAIN_DATA_DIR = 'C:/Users/yeyiy/Downloads/Dogs vs Cats/maybe'
TEST_DATA_DIR = 'C:/Users/yeyiy/Downloads/Dogs vs Cats/maybe'

def create_image_list():
    train_input = []
    train_images = []
    train_labels = []
    test_images = []
    for img in tqdm(os.listdir(TRAIN_DATA_DIR)):
        images = os.path.join(TRAIN_DATA_DIR, img)
        train_images.append(images)
#         display(Image.open(TRAIN_DATA_DIR+'/'+img))

        labels = img.split('.')[0]
        if labels == 'cat':
            train_labels.append(0)
        else:
            train_labels.append(1)
            
    for img in tqdm(os.listdir(TEST_DATA_DIR)):
        images = os.path.join(TEST_DATA_DIR, img)
        test_images.append(images)
            
    #         images = cv2.imread(images, cv2.IMREAD_COLOR)
#         images = image.load_img(images, target_size=(10,10))
#         print(type(images))
#         images = image.img_to_array(images)
    
    
    train_input = np.array([train_images, train_labels])
    train_input = train_input.transpose()
    np.random.shuffle(train_input)
    np.save('train_input.npy', train_input)
    
    t_images, v_images, t_labels, v_labels = train_test_split(
        train_images, train_labels, test_size=0.20, random_state=0)
    
    return t_images, t_labels, v_images, v_labels, test_images
    

### 生成器输出模块

In [73]:
def make_batch(i_images, i_labels, image_size):

    input_images = []
    
    for i_img in i_images:
#       images = cv2.imread(images, cv2.IMREAD_COLOR)
        images = image.load_img(i_img, target_size=(1,image_size,image_size, 3))
        images = image.img_to_array(images)
        input_images.append(images)
    input_images = np.array(input_images)
    
    # one hot eencoding
    input_labels = np_utils.to_categorical(i_labels)
    
    
    # 图像预处理
    datagen = image.ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        horizontal_flip=True,
        vertical_flip=True)
    
    datagen.fit(t_images, seed=10)
    train_batch = datagen.flow(input_images, input_labels,
                                 batch_size=64,shuffle=True,
                                 seed=10)
    
#     for x,y in train_batch:
#         print(y)
    
    return train_batch



In [ ]:
def make_test_batch(i_images, image_szie):
    input_images = []
    
    for i_img in i_images:
#       images = cv2.imread(images, cv2.IMREAD_COLOR)
        images = image.load_img(i_img, target_size=(1,image_size,image_size, 3))
        images = image.img_to_array(images)
        input_images.append(images)
    input_images = np.array(input_images)
    
    # 图像预处理
    datagen = image.ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        horizontal_flip=True,
        vertical_flip=True)
    
    datagen.fit(t_images, seed=10)
    test_batch = datagen.flow(input_images,shuffle=False,batch_size=64)
    
#     for x,y in train_batch:
#         print(y)
    
    return test_batch

### 结果可视化模块

In [ ]:
def show_history(t_history,train,valid):
    plt.plot(t_history.history[train])
    plt.plot(t_history.history[valid])
    plt.title('Xception Model '+ train +' History')
    plt.y_label(train)
    plt.x_label('Epoch')
    plt.legend(['train '+train, 'valid '+valid], loc='upper left')
    plt.show()
    

# 建立模型
使用预训练过的模型，使用迁移学习的思想进行模型的建立，并试图进行模型融合训练，使用到的模型有
- Inception v3
- InceptionResNetV2
- Xception

## Xception模型

In [ ]:
from kears.applications.Xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
from keras.optimizers import SGD
from IPython.display import SVG
import h5py as h5py

In [ ]:
model_xception = Xception(include_top=False, weights='imagenet', input_shape=(299,299,3))

for layer in model_xception.layers:
    layer.trainable = False
    
x = GlobalAveragePooling2D()(model_xception.output)

model1 = Flatten(name='flatten')(x)
model1 = Dense(4096, activation='relu', name='fc1')(model1)
model1 = Dense(4096, activation='relu', nmae='fc2')(model1)
model1 = Dropout(0.5)(model1)
model1 = Dense(2, activation='softmax', name='prediction')(model1)

model_xception_pred = Model(model_xception.input, model1, name='xception')


- 选择损失函数(loss)为：categorical_crossentropy  
- 选择优化器(optimizer)为：Nadam() 
- 选择评价函数为：accuracy

In [ ]:
model_xception_pred.compile(loss='categorical_crossentropy', optimizer=Nadam(), metrics=['accuracy'])

### 训练模型

In [ ]:
epochs = 10
image_size = 299

t_images, t_labels, v_images, v_labels, test_images = create_image_list()
x_t_generator = make_batch(t_images, t_labels, image_size)
x_v_generator = make_batch(v_images, v_labels, image_size)
x_test_generator = make_test_batch(test_images, image_size)
xception_history = model_xception_pred.fit_generator(x_t_generator, epochs=epochs, verbose=1, validation_data=x_v_generator)

### 模型可视化

In [ ]:
SVG(model_to_dot(model_xception_pred, show_shapes=True).create(prog="dot", format='svg'))

### 执行结果可视化

In [ ]:
# accuracy 可视化
show_history(xception_history, 'acc', 'val_acc')

In [ ]:
# loss 可视化
show_history(xception_history, 'loss', 'val_loss')

## 建立混合模型

### Xception 模型

In [ ]:
model_xception = Xception(include_top=False, weights='imagenet', input_shape=(299,299,3))
x = GlobalAveragePooling2D()(model_xception.output)
xception_model = Model(model_xception.input, x)

xception_train = xception_model.predict_generate(x_t_generator)
xception_valid = xception_model.predict_generate(x_v_generator)
xception_test = xception_model.predict_generator(x_test_generator)

# 将特征向量写入数据文件
with h5py.File("gap_Xception.h5") as h:
    h.create_dataset("train", data=xception_train)
    h.create_dataset("valid", data=xception_valid)
    h.create_dataset("test", data=xception_test)
    h.create_dataset("t_label", data=x_t_generator.classes)
    h.create_dataset("v_label", data=x_v_generator.classes)


### Inception v3 模型

In [ ]:
model_inception_v3 = InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3))
y = GlobalAveragePooling2D()(model_inception_v3.output)
inceptionV3_model = Model(model_inception_v3.input, y)

inceptionV3_train = inceptionV3_model.predict_generate(x_t_generator)
inceptionV3_valid = inceptionV3_model.predict_generate(x_v_generator)
inceptionV3_test = inceptionV3_model.predict_generator(x_test_generator)

# 将特征向量写入数据文件
with h5py.File("gap_InceptionV3.h5") as h:
    h.create_dataset("train", data=inceptionV3_train)
    h.create_dataset("valid", data=inceptionV3_valid)
    h.create_dataset("test", data=inceptionV3_test)
    h.create_dataset("t_label", data=x_t_generator.classes)
    h.create_dataset("v_label", data=x_v_generator.classes)

### InceptionResNetV2 模型

In [ ]:
model_inceptionresnetv2 = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(299,299,3))
z = GlobalAveragePooling2D()(model_inceptionresnetv2.output)
inceptionresnetv2_model = Model(model_inceptionresnetv2.input, y)

inceptionresnetv2_train = inceptionresnetv2_model.predict_generate(x_t_generator)
inceptionresnetv2_valid = inceptionresnetv2_model.predict_generate(x_v_generator)
inceptionresnetv2_test = inceptionresnetv2_model.predict_generator(x_test_generator)

# 将特征向量写入数据文件
with h5py.File("gap_InceptionResNetV2.h5") as h:
    h.create_dataset("train", data=inceptionresnetv2_train)
    h.create_dataset("valid", data=inceptionresnetv2_valid)
    h.create_dataset("test", data=inceptionresnetv2_test)
    h.create_dataset("t_label", data=x_t_generator.classes)
    h.create_dataset("v_label", data=x_v_generator.classes)

### 从文件中读取特征向量

In [ ]:
train_images = []
train_labels = []
valid_images = []
valid_labels = []
test_data = []

h5File = ["gap_Xception.h5", "gap_InceptionV3.h5", "gap_InceptionResNetV2.h5"]
for filename in h5File:
    for h5py.File(filename, 'r') as h:
        train_images.append(np.array(h['train']))
        train_labels.append(np.array(h['t_label']))
        valid_images.append(np.array(h['valid']))
        valid_labels.append(np.array(h['v_label']))
        test_data.append(np.array(h['test']))
        
# 将list竖直拼接
train_images = np.concatenate(train_images, axis=1)
train_labels = np.concatenate(train_labels, axis=1)
valid_images = np.concatenate(valid_images, axis=1)
valid_labels = np.concatenate(valid_labels, axis=1)
test_data = np.concatenate(test_data, axis=1)

train_images, train_labels = shuffle(train_images, train_labels)
valid_images, valid_labels = shuffle(valid_images, valid_labels)
valid_data = np.array(tuple([valid_images, valid_labels]))
valid_data = valid_data.transpose()


In [ ]:
train_input = Input(train_images.shape[1:])

model = Dropout(0.5)(train_input)
model = Dense(1, activation='sigmoid')(model)

new_model = Model(train_input, model)

In [ ]:
new_model.compile(loss='categorical_crossentropy', optimizer=Nadam(), metrics=['accuracy'])

In [ ]:
new_model.fit(train_images, train_labels, batch_size=128, epochs=10, verbose=2, validation_data=valid_data)

## 测试

In [ ]:
test_pred = new_model.predict(test_data, verbose=1)
test_pred = test_pred.clip(min=0.005, max=0.995)

file = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(TEST_DATA_DIR, (299, 299),
                                        shuffle=False, batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1 : fname.rfind('.')])
    file.set_value(index-1, 'label', test_pred[i])
    
file.to_csv('pred.csv', index=None)
file.head(10)

# _1, _2, _3, _4, test_images = create_image_list()

# for filename in enumerate(test_images):
#     i = 0
#     filename_with_null = filename.replace(TEST_DATA_DIR+"/","")
#     index = file_name_with_null.split(".")[0]
#     file.set_value(index, 'label', test_pred[i])
